Baseline này dùng các components sau:
- Vistral-7b quantized 4bit trên Kaggle
- BGE-m3 embedding models
- Vector database (qdrant)


## Set up Embedding Component

In [ ]:
%%capture
!pip install -Uq FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.8 MB/s eta 0:00:00


In [ ]:
%%capture
# Use capture to hide lengthy download outputs
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [ ]:
answers = ['Các triệu chứng của người bị rắn độc cắn là gì? - Các triệu chứng của vết rắn cắn có nọc độc bao gồm:\nTại vết cắn: - Sưng, đau hoặc kích thước của vết cắn tăng lên nhanh chóng. - Đau nhức hoặc cảm giác nóng rát quanh vùng vết cắn. - Có thể xuất hiện các dấu hiệu khác như: bầm tím, chảy máu, phồng rộp, hoại tử da.\nTriệu chứng toàn thân: - Buồn nôn, nôn mửa. - Tiêu chảy. - Chóng mặt, hoa mắt. - Đau đầu. - Mệt mỏi, yếu ớt. - Khó thở. - Lo lắng, bồn chồn. - Hạ huyết áp. - Mất ý thức. - Liệt cơ.\nLưu ý: - Mức độ và loại triệu chứng sẽ khác nhau tùy thuộc vào loại rắn cắn, lượng nọc độc xâm nhập và sức khỏe của nạn nhân. - Một số trường hợp có thể không có bất kỳ triệu chứng nào trong vài giờ đầu tiên sau khi bị cắn. - Tuy nhiên, bất kỳ ai bị rắn cắn đều cần được chăm sóc y tế ngay lập tức, ngay cả khi họ không có triệu chứng.\nMột số lưu ý quan trọng: - Không tự ý xử lý vết cắn của rắn. - Không cắt rạch hoặc hút nọc độc ra khỏi vết cắn. - Không băng bó vết cắn quá chặt. - Giữ cho nạn nhân nằm yên tĩnh và đưa họ đến cơ sở y tế gần nhất càng nhanh càng tốt.',
 'Các triệu chứng bị rắn cắn sẽ xuất hiện trong bao lâu? - Tùy từng loài rắn cắn mà biểu hiện triệu chứng của vết cắn sẽ xuất hiện sớm hay muộn khác nhau. Triệu chứng có thể xuất hiện sau 10 phút đến 60 phút.',
 'Những bước đầu tiên cần làm khi bị rắn cắn là gì? - • Giữ bình tĩnh và không hoảng sợ.\n• Gỡ bỏ mọi đồ trang sức, vòng tay hoặc quần áo bó sát khỏi vùng bị cắn vì khi vết thương sưng có thể làm thắt chặt gây tổn thương mô.\n• Rửa sạch vết cắn bằng nước sạch\n• Bất động vùng bị cắn bằng nẹp hoặc băng.\n• Hạ thấp vùng bị cắn hơn so với tim.\n• Không rạch vết cắn hoặc cố gắng hút nọc độc.\n• Không cho nạn nhân bất cứ thức uống gì ngoài nước lọc.\n• Theo dõi các triệu chứng của nạn nhân và đưa họ đến bệnh viện hoặc cơ sở y tế gần nhất ngay lập tức.',
 'Có phải luôn luôn cần phải đến bệnh viện sau khi bị rắn cắn? - Không, không phải lúc nào cũng cần đến bệnh viện sau khi bị rắn cắn. Chỉ cần đến bệnh viện trong những trường hợp sau: Khi bị rắn cắn vào vùng đầu, cổ, mặt, tay hoặc chân. - Khi vết cắn gây đau, sưng, đỏ và nóng. - Khi nạn nhân bị chóng mặt, buồn nôn, hoặc khó thở. - Khi nạn nhân bị tê liệt hoặc yếu ớt.',
 'Có biện pháp nào tại gia nào có tác dụng chữa rắn cắn không? - Không có biện pháp khắc nào được chứng minh là có hiệu quả cả. Tuy nhiên, có một số biện pháp sơ cứu có thể làm chậm quá trình ngộ độc trong khi chờ được đưa đến cơ sở y tế để điều trị.',
 'Sử dụng loại huyết thanh nào cho các loài rắn khác nhau? - Các loại huyết thanh kháng nọc rắn được sử dụng tùy thuộc vào loài rắn:\n• Nọc rắn hổ mang: Huyết thanh kháng độc nọc rắn hổ mang đa giá (Polyvalent cobra antivenom)\n• Nọc rắn lục: Huyết thanh kháng độc nọc rắn lục (Green pit viper antivenom)\n• Nọc rắn hổ chúa: Huyết thanh kháng độc nọc rắn hổ chúa (King cobra antivenom)\n• Nọc rắn cạp nong: Huyết thanh kháng độc nọc rắn cạp nong (Krait antivenom)\n• Nọc rắn biển: Huyết thanh kháng độc nọc rắn biển (Sea snake antivenom)',
 'Phải mất bao lâu để phục hồi sau một vết rắn cắn? - Thời gian hồi phục sau vết rắn cắn thường phụ thuộc vào các yếu tố như loại rắn, lượng nọc độc tiêm vào, vị trí vết cắn và sức khỏe tổng thể của nạn nhân. Trong hầu hết các trường hợp, thời gian hồi phục có thể dao động từ vài ngày đến vài tuần. Đối với các vết cắn nghiêm trọng, thời gian hồi phục có thể kéo dài hơn hoặc có thể để lại di chứng lâu dài, chẳng hạn như hoại tử mô, tổn thương thần kinh hoặc mất chức năng.',
 'Tôi có thể ngăn chặn rắn xâm nhập vào tài sản của mình như thế nào? - Để ngăn ngừa rắn xâm nhập vào nhà bạn, bạn có thể thực hiện một số biện pháp như sau: - Loại bỏ nguồn thức ăn cho rắn: Rắn thường bị thu hút bởi chuột, gián và các loài gặm nhấm nhỏ khác. Để ngăn chặn rắn xâm nhập, bạn cần loại bỏ nguồn thức ăn của chúng bằng cách bít kín các lỗ hổng trên tường và sàn nhà, dọn dẹp rác thải và thức ăn thừa, cũng như sử dụng bẫy để diệt chuột. -Lắp đặt hàng rào chắn rắn: Hàng rào chắn rắn là một biện pháp hiệu quả để ngăn chặn rắn xâm nhập vào nhà của bạn. Bạn có thể lắp đặt hàng rào lưới mắt cáo hoặc hàng rào kim loại xung quanh chu vi của ngôi nhà, đảm bảo rằng hàng rào được chôn sâu xuống đất ít nhất 30 cm để ngăn rắn đào hang. - Bít kín các lỗ hổng: Rắn có thể chui qua những lỗ hổng nhỏ, vì vậy bạn cần bít kín tất cả các khoảng trống và lỗ hổng xung quanh nhà, đặc biệt là ở các khu vực như đường ống, lỗ thông hơi và cửa sổ. Bạn có thể sử dụng vật liệu như lưới thép, bọt hoặc xi măng để bít kín các lỗ hổng. - Giữ cho sân sạch sẽ: Rắn thích trú ẩn trong các đống đổ nát và rác thải, vì vậy hãy giữ cho sân của bạn sạch sẽ và không có nơi trú ẩn cho rắn. Dọn dẹp cỏ dại, cắt tỉa cây cối và loại bỏ các vật dụng không cần thiết có thể trở thành nơi trú ẩn cho rắn. - Kiểm tra định kỳ: Kiểm tra định kỳ quanh nhà để tìm kiếm dấu hiệu của rắn, chẳng hạn như phân, da lột và dấu vết di chuyển. Nếu bạn phát hiện bất kỳ dấu hiệu nào của rắn, hãy liên hệ với dịch vụ bắt rắn chuyên nghiệp để được hỗ trợ.',
 'Một số cách tự nhiên nào để ngăn rắn vào sân nhà tôi? - Một số cách tự nhiên để ngăn rắn vào sân nhà bạn:Giữ cho sân vườn sạch sẽ, gọn gàng, loại bỏ những nơi ẩn náu tiềm ẩn cho rắn như: đống gỗ, lùm cây, … - Lắp đặt lưới thép hoặc hàng rào chắn xung quanh nhà để ngăn rắn xâm nhập. -Lưu ý: - Tránh làm tổn thương hoặc tiêu diệt rắn, vì chúng đóng vai trò quan trọng trong hệ sinh thái. - Nếu bạn phát hiện rắn trong nhà, hãy liên hệ với dịch vụ bắt rắn chuyên nghiệp Viet Snake Rescuer để được hỗ trợ.\nĐiều quan trọng là bạn cần kiên trì thực hiện các biện pháp phòng ngừa để ngăn rắn xâm nhập vào sân nhà và đảm bảo an toàn cho bản thân và gia đình.',
 'Tôi nên gọi thợ chuyên nghiệp nếu tìm thấy rắn trong nhà hay không? - Có, bạn nên Liên hệ ngay đến Viet Snake Rescuer  nếu bạn tìm thấy một con rắn trong nhà của mình.',
 'Có mối nguy hiểm nào đối với vật nuôi từ loài rắn thường tìm thấy trong sân không? - Có nhiều nguy hiểm đối với vật nuôi từ những con rắn thường thấy trong các sân vườn.\n• Độc: Nhiều loại rắn có nọc độc có thể gây tử vong cho vật nuôi, tùy thuộc vào loại rắn và kích thước của vật nuôi.\n• Thắt cổ: Một số loài rắn, chẳng hạn như trăn và trăn khổng lồ, có thể quấn chặt quanh vật nuôi, khiến chúng ngạt thở.\n• Truyền bệnh: Rắn có thể mang các loại vi khuẩn và ký sinh trùng có thể gây bệnh cho vật nuôi.\n• Sợ hãi và căng thẳng: Ngay cả khi rắn không tấn công, chúng có thể gây sợ hãi và căng thẳng cho vật nuôi, ảnh hưởng đến sức khỏe của vật nuôi.',
 'Tôi nên làm gì khi phát hiện ra tổ rắn gần nhà? - Nếu bạn tìm thấy một tổ rắn gần nhà của mình, bạn nên làm như sau:\n1. Giữ bình tĩnh và tránh tiếp cận tổ.\n2. Quan sát tổ từ xa để xác định xem có rắn ở bên trong không.\n3. Gọi cho Viet Snake Rescuer  để được trợ giúp.\n4. Không cố gắng tự mình di chuyển hoặc tiêu diệt tổ rắn.\n5. Cảnh báo những người khác trong khu vực về tổ rắn và khuyên họ nên cẩn thận.\n6. Tránh để trẻ em và vật nuôi đến gần tổ rắn.',
 'Rắn có phải là mối đe dọa đối với mèo hoặc chó không? - Có, rắn có thể gây nguy hiểm cho cả mèo và chó nuôi.',
 'Làm thế nào để tôi có thể huấn luyện thú cưng của mình tránh xa rắn? - Huấn luyện vật nuôi của bạn tránh xa rắn:\n1. Giữ rắn sống hoặc đóng băng ngoài tầm với: Điều này giúp vật nuôi của bạn quen với mùi và sự hiện diện của rắn, nhưng không gây nguy hiểm.\n2. Sử dụng lệnh “không”: Khi vật nuôi của bạn tiếp cận một con rắn, hãy nói “không” với giọng nghiêm khắc. Lặp lại lệnh này mỗi khi chúng tiếp cận một con rắn.\n3. Khen ngợi hành vi mong muốn: Khi vật nuôi của bạn tuân theo lệnh “không” hoặc tránh xa rắn, hãy khen ngợi và thưởng cho chúng. Điều này giúp chúng liên kết hành vi mong muốn với phần thưởng.\n4. Dây xích và huấn luyện tuân lệnh: Mang theo vật nuôi của bạn đi dạo bằng dây xích và huấn luyện chúng tuân lệnh “ở lại” hoặc “bên cạnh”. Điều này giúp bạn kiểm soát chúng khi đi dạo ở những khu vực có thể có rắn.\n5. Huấn luyện phản xạ: Sử dụng một công cụ phát ra tiếng ồn như còi hoặc bình xịt để tạo ra tiếng ồn lớn khi vật nuôi của bạn tiếp cận một con rắn. Tiếng ồn này sẽ làm chúng giật mình và liên kết rắn với thứ gì đó khó chịu.\n6. Đưa vật nuôi đi thú y: Nếu vật nuôi của bạn vẫn tiếp cận rắn bất chấp việc huấn luyện, hãy đưa chúng đi khám bác sĩ thú y. Có thể có vấn đề về hành vi hoặc sức khỏe tiềm ẩn khiến chúng không thể tuân theo lệnh.',
 'Những dấu hiệu nào cho thấy thú cưng của bạn bị rắn cắn? - Những dấu hiệu cho thấy vật nuôi bị rắn cắn.\n• Vết thương (Hai vết đâm nhỏ từ răng nanh của rắn) có thể hoặc không chảy máu.\n• Chỗ bị cắn sưng tấy, đau và có thể đổi màu (từ đỏ đến tím đến đen).\n• Các dấu hiệu sốc (yếu, chóng mặt, khó thở, nôn mửa).\n• Các vấn đề đông máu (chảy máu mũi, nướu hoặc miệng).\n• Liệt (bị tê liệt ở một hoặc nhiều chân).\n• Các vấn đề về thần kinh (co giật, run rẩy hoặc chậm chạp).\n• Vấn đề tiêu hóa (tiêu chảy, nôn mửa).\n• Đau và đau nhức sau khi bị cắn.',
 'Có loài rắn nào an toàn để nuôi làm thú cưng không? - Có một số loài rắn được coi là an toàn để nuôi làm thú cưng. Chúng thường có kích thước nhỏ, không có nọc độc và tính cách hiền lành. Bạn nên cân nhắc về tính hợp pháp của việc nuôi thú cưng bò sát, bao gồm tính minh bạch nguồn gốc, xuất sứ của con vật, cũng như sự cho phép từ pháp luật. Tuyệt đối không hưởng ứng việc tiêu thụ, nuôi nhốt động vật hoang dã.',
 'Quy định pháp luật về việc nuôi rắn làm thú cưng ở khu vực của tôi là gì? - Có rất nhiều loài rắn nằm trong bảo vệ của pháp luật Việt Nam và quốc tế. Vì vậy trước khi ra quyết định mua thú cưng là rắn, bạn phải nghiên cứu thật kĩ về nguồn gốc xuất xứ, thông tin người bán nhằm đảm bảo bạn không tiếp tay cho việc tiêu thụ sản phẩm động vật hoang dã.',
 'Tỉ lệ người sợ rắn (ophidiophobia) phổ biến như thế nào? - Ophidiophobia, nỗi sợ hãi với rắn, là một trong những nỗi sợ phổ biến nhất trên thế giới, ảnh hưởng đến khoảng một trong ba người.',
 'Có một số chiến lược nào để vượt qua nỗi sợ rắn? - Các chiến lược để vượt qua nỗi sợ rắn\n1. Tìm hiểu về rắn\n• Đọc sách, xem phim tài liệu và nói chuyện với các chuyên gia về rắn để có kiến thức đúng đắn về loài vật này.\n• Tìm hiểu về các loại rắn khác nhau, đặc điểm, hành vi và mức độ nguy hiểm của chúng.\n• Hiểu biết về rắn sẽ giúp bạn giải tỏa nỗi sợ hãi và hiểu rằng chúng không phải là những sinh vật đáng sợ như bạn tưởng tượng.\n2. Tiếp xúc dần dần\n• Bắt đầu bằng việc tiếp xúc với hình ảnh về rắn, chẳng hạn như ảnh hoặc video.\n• Tham gia các hội nhóm trang mạng XH có liên quan đến động vật - tự nhiên để có thể trao đổi với mọi người nhiều hơn về loài rắn\n• Tham gia các hội nhóm nuôi vật cưng về bò sát để được trao đổi kĩ hơn và hiểu biết sâu hơn cụ thể hơn về từng loài rắn\n4. Phơi nhiễm \n\nTham quan những nơi trưng bày mẫu vật rắn như các bảo tàng tự nhiên\nTham gia buổi gặp mặt của các CLB nuôi thú cưng bò sát\nXin phép những người nuôi hướng dẫn bạn cách tiếp xúc và tương tác với rắn\n\n5. Tìm kiếm sự hỗ trợ\n• Nếu bạn gặp khó khăn trong việc vượt qua nỗi sợ hãi, hãy tìm kiếm sự hỗ trợ từ bạn bè, gia đình hoặc chuyên gia trị liệu.\n• Họ có thể cung cấp sự khuyến khích, động viên và hỗ trợ bạn trong quá trình này.\n• Tham gia các nhóm hỗ trợ hoặc trị liệu có thể giúp bạn kết nối với những người có cùng nỗi sợ hãi và học hỏi từ kinh nghiệm của họ.\nLưu ý:\n• Hãy kiên nhẫn và kiên trì. Vượt qua nỗi sợ hãi mất thời gian và nỗ lực.\n• Đừng vội nản lòng nếu bạn cảm thấy sợ hãi trở lại. Đây là một phần của quá trình.\n• Nếu bạn gặp bất kỳ tình huống nguy hiểm nào liên quan đến rắn, hãy tìm kiếm sự giúp đỡ chuyên nghiệp ngay lập tức.',
 'Làm thế nào để có thể giúp một người đang hoảng sợ sau khi thấy con rắn? - Cách giúp người bị hoảng loạn sau khi nhìn thấy rắn:\n1. Giữ bình tĩnh và an toàn:\n• Hãy bình tĩnh và trấn an người đó rằng họ đã an toàn.\n• Kiểm tra xem có bất kỳ vết thương nào không và gọi dịch vụ cấp cứu nếu cần thiết.\n2. Tạo không gian an toàn:\n• Di chuyển người đó ra khỏi khu vực có rắn nếu có thể.\n• Cung cấp cho họ một không gian yên tĩnh và dễ chịu.\n3. Giúp họ thở chậm lại:\n• Hướng dẫn họ hít thở sâu bằng cách hít vào bằng mũi và thở ra bằng miệng.\n• Bạn cũng có thể sử dụng kỹ thuật hít thở túi giấy: phủ một chiếc túi giấy lên miệng và mũi của họ và hướng dẫn họ thở vào và ra từ từ.\n4. Nghe và trấn an:\n• Cho người đó thời gian để nói chuyện về những gì đã xảy ra.\n• Nghe thấu hiểu và không phán xét.\n• Trấn an họ rằng phản ứng của họ là bình thường và họ không đơn độc.\n\n6. Tìm kiếm sự trợ giúp chuyên nghiệp nếu cần thiết:\n• Nếu người đó tiếp tục bị hoảng loạn dữ dội hoặc lo lắng liên tục, hãy đề nghị tìm kiếm sự trợ giúp chuyên nghiệp từ bác sĩ tâm lý hoặc nhà trị liệu.\nGhi chú:\n• Không cố gắng tiếp cận con rắn nếu bạn không được đào tạo để làm như vậy.\n\n• Nếu có thể, hãy chụp ảnh con rắn để có thể nhận dạng loài của nó (nếu cần thiết cho mục đích y tế).',
 'Có đúng tất cả các loài rắn đều hung dữ và sẽ đuổi theo con người không? - Không có loài rắn hung dữ, chỉ có loài rắn có bản năng tự vệ cao. Rắn không bao giờ chủ động rượt đuổi vì muốn tấn công con người.',
 'Liệu rằng rắn có thật sự tấn công con người nếu bạn giẫm phải chúng? - Không, rắn thường không tấn công người nếu bạn vô tình giẫm lên chúng.Mặc dù vậy,tốt nhất bạn nên tránh giẫm lên rắn vì giẫm lên chúng có thể khiến chúng bị thương hoặc sợ hãi,làm chúng phản kháng tự vệ. Nếu bạn nhìn thấy rắn,hãy bình tĩnh lùi ra và tránh đe doạ nó.',
 'Liệu rắn có thể nghe thấy âm thanh hay chúng chỉ dựa vào rung động? - Rắn có thể nghe thấy âm thanh nhưng không phải qua tai. Chúng cảm nhận âm thanh thông qua các thụ thể đặc biệt ở bụng và xương hàm. Các thụ thể này giúp chúng nhận biết được rung động trong không khí và trên mặt đất, cho phép chúng xác định vị trí con mồi hoặc phát hiện những kẻ săn mồi tiềm ẩn.',
 'Một con rắn đã chết vẫn có thể cắn không? - Đúng, dù rắn đã chết nhưng một số phản ứng thần kinh vẫn có thể còn hoạt động cho phép rắn vẫn có thể tạo ra hành động cắn trả một cách vô điều kiện',
 'Có đúng là nếu cắt đôi một con rắn thì cả hai đầu sẽ sống tiếp không? - Câu trả lời là: Không.\nKhi một con rắn bị cắt làm đôi, cả hai đầu sẽ không thể sống được.',
 'Có phải có loài rắn nào đang bị đe dọa hay tuyệt chủng không? - Có nhiều loài rắn đang bị tuyệt chủng hoặc bị đe dọa.',
 'Rắn đóng vai trò gì trong môi trường? - Trong tự nhiên, rắn đóng vai trò là loài săn mồi, kiểm soát số lượng các loài động vật gặm nhấm, giúp cân bằng hệ sinh thái. Đối với con người, nọc độc của một số loài rắn còn được nghiên cứu, ứng dụng trong y học.',
 'Làm sao tôi có thể bảo vệ quần thể rắn ở khu vực của mình? - Có thể giúp bảo vệ quần thể rắn trong khu vực của mình bằng những cách sau:\n1. Không giết rắn: Rắn đóng vai trò quan trọng trong hệ sinh thái, giúp kiểm soát quần thể động vật gặm nhấm.\n2. Tôn trọng không gian của rắn: Nếu bạn nhìn thấy rắn, hãy để nó đi theo đường của nó. Đừng cố bắt hoặc giết nó.\n3. Tạo môi trường sống cho rắn: Cung cấp nơi trú ẩn và thức ăn cho rắn bằng cách duy trì các khu vực cỏ dài, đá và bụi rậm.\n4. Giáo dục những người khác: Chia sẻ kiến \u200b\u200bthức của bạn về tầm quan trọng của rắn với những người khác. Càng nhiều người hiểu về những lợi ích của rắn, thì càng có nhiều khả năng họ sẽ được bảo vệ.\n5. Hỗ trợ các nhóm bảo tồn: Có nhiều tổ chức đang làm việc để bảo vệ rắn. Bạn có thể hỗ trợ công việc của họ bằng cách hiến tặng thời gian, tiền bạc hoặc cả hai.',
 'Có đạo đức không khi giết một con rắn mà tôi tìm thấy trên tài sản của mình? - Tùy thuộc vào từng trường hợp cụ thể mà có thể là hợp lý hoặc không hợp lý về mặt đạo đức để giết một con rắn mà bạn tìm thấy trên tài sản của mình. Một số yếu tố cần cân nhắc bao gồm: - Loại rắn: Nếu con rắn có nọc độc hoặc có nguy cơ gây nguy hiểm cho bạn hoặc gia đình, thì việc giết nó có thể được coi là hợp lý về mặt đạo đức để bảo vệ tính mạng và sự an toàn của chính bạn. - Mối đe dọa đối với vật nuôi hoặc tài sản: Nếu con rắn đang đe dọa vật nuôi hoặc tài sản của bạn, thì việc giết nó cũng có thể được coi là hợp lý về mặt đạo đức để bảo vệ lợi ích của bạn. - Tình trạng bảo tồn: Nếu con rắn là một loài được bảo vệ hoặc có nguy cơ tuyệt chủng, thì việc giết nó có thể bị coi là không hợp lý về mặt đạo đức. - Giải pháp thay thế: Có những giải pháp thay thế để giết rắn, chẳng hạn như bắt và thảNhững giải pháp thay thế này có thể được ưu tiên hơn việc giết rắn, đặc biệt là đối với những loài vô hại hoặc có nguy cơ tuyệt chủng. Cuối cùng, quyết định có giết hay không giết một con rắn mà bạn tìm thấy trên tài sản của mình là một quyết định cá nhân. Tuy nhiên, điều quan trọng là phải cân nhắc đầy đủ các yếu tố có liên quan và đưa ra quyết định dựa trên các nguyên tắc đạo đức.',
 'Có tổ chức nào dành riêng cho bảo tồn rắn ở Việt Nam không? - Có Viet Snake Rescuer',
 'Rắn tượng trưng cho điều gì trong các nền văn hóa khác nhau trên thế giới? - Rắn có thể là biểu tượng cho nhiều thứ khác nhau, tùy thuộc vào nền văn hóa.\n• Trong văn hóa Ai Cập cổ đại, rắn được coi là biểu tượng của sự tái sinh và bất tử.\n• Trong văn hóa Trung Quốc, rắn được liên kết với sự khôn ngoan, sức mạnh và may mắn.\n• Trong văn hóa Ấn Độ, rắn được thờ phụng như một vị thần và được coi là biểu tượng của dục vọng và sự bảo vệ.\n• Trong văn hóa Cơ đốc giáo, rắn thường được coi là biểu tượng của sự cám dỗ và tội lỗi.\n• Trong văn hóa phương Tây, rắn thường được liên kết với sự nguy hiểm, thủ đoạn và xảo quyệt.',
 'Có bất kỳ tín ngưỡng tôn giáo nào liên quan đến loài rắn không? - Có rất nhiều tín ngưỡng tôn giáo liên quan đến rắn. Trong Ấn Độ giáo, rắn thần Nagaraja được tôn thờ như một vị thần bảo vệ và là biểu tượng của sự tái sinh. Trong Ai Cập cổ đại, thần rắn Apep được coi là hiện thân của bóng tối và hỗn loạn. Trong truyền thống Kitô giáo, rắn là biểu tượng của sự cám dỗ và tội lỗi, nhưng cũng tượng trưng cho sự chữa lành và phục sinh. Trong Phật giáo, rắn là biểu tượng của sự khôn ngoan và từ bi.',
 'Rắn xuất hiện như thế nào trong thần thoại và văn hóa dân gian Việt Nam? - Trong thần thoại và văn hóa dân gian Việt Nam, rắn đóng vai trò quan trọng và có nhiều hình tượng khác nhau:\n1. Rắn thần/Long: - Biểu tượng của sự quyền lực, uy nghi và siêu nhiên. - Thường được gắn với các vị thần như Long Vương, Thủy Thần, Thành Hoàng. - Người ta tin rằng rắn thần có thể ban phúc lành, che chở cho con người và bảo vệ đất nước.\n2. Rắn tiên: - Được miêu tả là những con rắn có màu trắng hoặc xanh, thân dài và có phép thuật. - Thường xuất hiện trong các truyền thuyết, truyện cổ tích, gắn liền với các nhân vật anh hùng hoặc thánh nhân. - Rắn tiên có thể biến hình, giúp đỡ con người vượt qua khó khăn, chiến thắng kẻ thù.\n3. Rắn thần giữ của: - Những con rắn khổng lồ được giao nhiệm vụ trông giữ kho báu hoặc của cải quý giá. - Theo truyền thuyết, nếu ai dám lấy trộm của sẽ bị rắn thần trừng phạt nặng nề.\n4. Rắn tinh: - Những con rắn có khả năng tu luyện thành tinh, có phép thuật cao cường. - Thường được miêu tả là những kẻ xảo trá, gian xảo, có thể biến hình để lừa gạt con người.\n5. Rắn độc: - Có nhiều loại rắn độc được mô tả trong thần thoại và văn hóa dân gian, như rắn hổ mang, rắn cạp nong, rắn lục. - Người ta rất sợ rắn độc vì chúng có thể gây chết người chỉ với một vết cắn.\n6. Rắn là biểu tượng của: - Sức mạnh, uy quyền - Sự khôn ngoan, mưu mẹo - Sự quyến rũ, bí ẩn - Sự xảo trá, gian manh\nTrong văn học dân gian Việt Nam, hình ảnh con rắn còn được sử dụng để ẩn dụ, tượng trưng cho nhiều điều khác nhau, chẳng hạn như: - Sự chuyển đổi, biến đổi - Sự nguy hiểm, hiểm họa - Sự cám dỗ, quyến rũ',
 'Có nền văn hóa nào tôn sùng rắn trong khi những nền văn hóa khác lại sợ hãi chúng không? - Đúng vậy, có rất nhiều nền văn hóa trên thế giới có quan điểm trái ngược nhau về rắn. Một số nền văn hóa tôn sùng rắn, coi chúng là biểu tượng thiêng liêng và quyền lực, trong khi những nền văn hóa khác lại sợ hãi và căm ghét chúng.\nTôn sùng rắn:\n• Ấn Độ:Rắn đóng vai trò quan trọng trong Ấn Độ giáo và Hindu giáo. Thần Vishnu được miêu tả là ngủ trên con rắn Shesha, và nhiều vị thần khác được trang trí bằng rắn. Rắn hổ mang vua được coi là linh thiêng và được tôn thờ ở một số nơi.\n• Ai Cập:Trong thần thoại Ai Cập, rắn được coi là biểu tượng của sự bảo vệ, chữa bệnh và hoàng gia. Nữ thần Wadjet, được mô tả là một con rắn hoặc phụ nữ có đầu rắn, là người bảo vệ pharaoh. Hình ảnh về Rắn trong thần thoại Ai CậpMở trong cửa sổ mới\n• Văn hóa Maya:Người Maya tôn thờ rắn thần Kukulkan, được cho là vị thần sáng tạo và mưa. Hình ảnh rắn được tìm thấy trong nhiều di tích Maya, và rắn đóng vai trò quan trọng trong các nghi lễ tôn giáo. Hình ảnh về Rắn trong văn hóa MayaMở trong cửa sổ mới\n• Văn hóa Yoruba:Trong tín ngưỡng Yoruba của Tây Phi, rắn được coi là biểu tượng của trí tuệ, sự chữa bệnh và sự biến đổi. Vị thần Olokun, thần cai quản đại dương, thường được miêu tả là một con rắn khổng lồ.\n Sợ hãi rắn:\n• Nhiều nền văn hóa phương Tây: Trong nhiều nền văn hóa phương Tây, rắn được coi là biểu tượng của sự ác quỷ và cám dỗ. Hình ảnh rắn trong Kinh thánh đóng vai trò quan trọng trong quan điểm này, vì rắn được cho là đã cám dỗ A-đam và Eva trong Vườn Địa đàng.\n• Một số nền văn hóa châu Á: Ở một số nền văn hóa châu Á, rắn được coi là biểu tượng của sự xui xẻo và cái chết. Ví dụ, ở Trung Quốc, số 4 được coi là con số không may mắn vì nó phát âm giống như từ “chết” trong tiếng Trung.\n• Một số nền văn hóa thổ dân: Một số nền văn hóa thổ dân trên thế giới cũng có niềm tin tiêu cực về rắn. Ví dụ, trong một số nền văn hóa thổ dân Úc, rắn được coi là hiện thân của linh hồn ma quỷ. Điều quan trọng cần lưu ý là những quan điểm về rắn khác nhau rất nhiều tùy thuộc vào từng nền văn hóa và cá nhân. Một số người có thể sợ rắn vì những trải nghiệm tiêu cực trong quá khứ, trong khi những người khác có thể tôn sùng chúng vì ý nghĩa tôn giáo hoặc văn hóa.\nNgoài ra, cách nhìn nhận rắn cũng có thể thay đổi theo thời gian. Ví dụ, ở châu  u, rắn từng được tôn sùng như một biểu tượng của sự chữa bệnh, nhưng sau đó lại bị gắn liền với hình ảnh phù thủy và ma thuật đen tối.\nNhìn chung, mối quan hệ giữa con người và rắn rất phức tạp và đa dạng. Sự tôn sùng hay sợ hãi rắn phụ thuộc vào nhiều yếu tố, bao gồm văn hóa, tôn giáo, kinh nghiệm cá nhân và niềm tin cá nhân.',
 'Ý niệm tiêu cực với rắn trong văn hóa Việt Nam xuất phát từ đâu? - Trong văn hóa Việt Nam, rắn thường gắn liền với những quan niệm tiêu cực vì các lý do sau:\n• Truyền thuyết và thần thoại: Trong truyền thuyết và thần thoại Việt Nam, rắn thường được miêu tả là sinh vật độc ác, xảo quyệt và có khả năng gây hại cho con người. Ví dụ, trong truyện cổ tích “Thạch Sanh”, rắn là kẻ thù chính, luôn tìm cách hãm hại Thạch Sanh.\n• Độc tính: Nhiều loài rắn ở Việt Nam có nọc độc, có thể gây thương tích nghiêm trọng hoặc thậm chí tử vong cho con người. Điều này làm tăng nỗi sợ hãi và ác cảm đối với rắn.\n• Sự ẩn núp và bí ẩn: Rắn là loài động vật có khả năng ẩn núp và di chuyển nhanh chóng, điều này khiến chúng trở nên khó đoán và dễ gây sợ hãi.\n• Liên hệ với thế giới bên kia: Trong văn hóa tâm linh Việt Nam, rắn thường được liên hệ với thế giới bên kia và những thế lực siêu nhiên. Người ta tin rằng rắn có khả năng giao tiếp với người chết và là sứ giả giữa hai thế giới.\n• Ảnh hưởng của tôn giáo: Trong một số tôn giáo, chẳng hạn như Phật giáo và Đạo giáo, rắn được coi là sinh vật đại diện cho cái ác, sự cám dỗ và tội lỗi. Điều này đã ảnh hưởng đến cách nhìn nhận của người Việt Nam về rắn.\nTừ truyền thông, phim ảnh: Chủ động dùng hình ảnh con rắn để làm vật ẩn dụ, gán ghép cho những ý nhiệm xấu xa, nham hiểm, độc ác.',
 'Rắn mang lại lợi ích gì cho con người hoặc môi trường? - Rắn mang lại nhiều lợi ích cho con người và môi trường, bao gồm:\nKiểm soát dịch hại: Rắn là động vật săn mồi đóng vai trò quan trọng trong việc kiểm soát các loài gặm nhấm, côn trùng và các loài gây hại khác. Bằng cách săn bắt những loài này, chúng giúp bảo vệ mùa màng, tài sản và sức khỏe con người.\nNguồn cung cấp thuốc men: Nọc độc của rắn đã được sử dụng trong y học từ hàng nghìn năm để điều trị một loạt các bệnh, bao gồm đau mãn, ung thư và bệnh tim. Nghiên cứu về nọc rắn đang diễn ra, với mục đích khám phá các ứng dụng y tế mới cho chất độc mạnh này.\nChỉ báo sức khỏe hệ sinh thái: Rắn là loài chỉ thị, có nghĩa là sự hiện diện hoặc vắng mặt của chúng có thể cung cấp thông tin về sức khỏe tổng thể của hệ sinh thái. Ví dụ, sự suy giảm quần thể rắn có thể là dấu hiệu của sự ô nhiễm, mất môi trường sống hoặc các vấn đề khác đang gây bất lợi cho hệ sinh thái.\nGiá trị thẩm mỹ và văn hóa: Rắn có thể là loài động vật đẹp và được tôn kính trong nhiều nền văn hóa trên thế giới. Chúng được đặc tả trong thần thoại, nghệ thuật và văn học, và một số loài được nuôi làm vật nuôi hoặc được trưng bày trong sở thú.\nDu lịch sinh thái: Khi du khách đến các khu vực có rắn, họ có thể đóng góp cho nền kinh tế địa phương và hỗ trợ các nỗ lực bảo tồn. Du lịch sinh thái liên quan đến rắn có thể thúc đẩy nhận thức về loài này, đồng thời tạo ra nguồn thu nhập kích thích bảo vệ chúng.\nBảo tồn đa dạng sinh học: Rắn đóng vai trò thiết yếu trong lưới thức ăn và giúp duy trì đa dạng sinh học. Chúng liên quan đến nhiều loài thực vật và động vật khác, và sự mất mát của chúng có thể tạo ra hiệu ứng gợn sóng đe dọa tính toàn vẹn của hệ sinh thái.\nBằng cách bảo vệ rắn và môi trường sống của chúng, chúng ta có thể tận hưởng những lợi ích này và đảm bảo tương lai của những loài hấp dẫn và quan trọng này.',
 'Rắn giúp kiểm soát quần thể loài gặm nhấm bằng cách nào? - Rắn giúp kiểm soát quần thể loài gặm nhấm bằng cách ăn chúng. Rắn là loài săn mồi đỉnh cao trong nhiều hệ sinh thái và chúng đóng một vai trò quan trọng trong việc giữ cho quần thể gặm nhấm không quá lớn. Gặm nhấm có thể gây ra nhiều vấn đề cho con người, bao gồm phá hoại mùa màng, ô nhiễm nguồn nước và lây lan bệnh tật. Bằng cách ăn gặm nhấm, rắn giúp bảo vệ con người và các nguồn tài nguyên của chúng.',
 'Nọc rắn có công dụng gì trong y học không? - Nọc rắn có thể được ứng dụng vào y học trong những trường hợp sau:\n• Chống đông máu: Nọc rắn chứa các chất có khả năng ức chế quá trình đông máu, do đó có thể được sử dụng làm thuốc chống đông trong phẫu thuật và điều trị tắc nghẽn mạch máu.\n• Giảm đau: Một số thành phần trong nọc rắn có tác dụng giảm đau mạnh mẽ, có thể được sử dụng để điều trị đau cấp tính và đau mãn.\n• Điều trị ung thư: Một số loại nọc rắn đã được nghiên cứu về tiềm năng chống lại tế bào ung thư. Các chất độc trong nọc rắn có thể làm tổn thương và tiêu diệt các tế bào ung thư mà không gây hại cho các tế bào khỏe mạnh.\n• Phát triển thuốc mới: Các chất độc trong nọc rắn có thể được nghiên cứu và phát triển thành các loại thuốc mới để điều trị nhiều bệnh khác nhau. Ví dụ, một số chất độc trong nọc rắn đã được sử dụng để phát triển các thuốc điều trị bệnh Alzheimer và bệnh tim.',
 'Có thể nghiên cứu về rắn dạy chúng ta điều gì về sức khỏe của con người không? - Nghiên cứu rắn có thể dạy chúng ta điều gì về sức khỏe con người? - Có rất nhiều điều chúng ta có thể học về sức khỏe con người bằng cách nghiên cứu rắn. Ví dụ, nghiên cứu về nọc rắn đã giúp chúng ta phát triển các loại thuốc mới để điều trị đau tim, đột quỵ và ung thư. Nghiên cứu về da rắn đã giúp chúng ta phát triển các loại vật liệu mới để sử dụng trong y học tái tạo. Và nghiên cứu về cách rắn di chuyển đã giúp chúng ta phát triển các loại robot mới có thể giúp người khuyết tật.\nNgoài khả năng cung cấp những hiểu biết về sức khỏe con người, nghiên cứu về rắn cũng có thể giúp chúng ta bảo tồn các loài bị đe dọa. Nghiên cứu về rắn có thể giúp chúng ta hiểu được cách bảo vệ các loài rắn khỏi tuyệt chủng và cách chúng ta có thể chung sống hòa bình với chúng.\nCác loài rắn là một phần quan trọng của hệ sinh thái, đóng vai trò trong kiểm soát loài gặm nhấm và các loài gây hại khác. Chúng cũng là nguồn thực phẩm quan trọng cho những loài động vật khác, chẳng hạn như đại bàng và sói. Bằng cách bảo vệ rắn, chúng ta đang giúp bảo vệ toàn bộ hệ sinh thái.\nNghiên cứu về rắn là một lĩnh vực hấp dẫn и hữu ích. Những gì chúng ta học được từ nghiên cứu về rắn có thể giúp chúng ta cải thiện sức khỏe, bảo tồn các loài bị đe dọa và hiểu thêm về thế giới tự nhiên.',
 'Thời điểm nào là tốt nhất để gọi chuyên gia đến bắt rắn? - Không có thời điểm nào hoàn toàn tốt nhất để gọi chuyên gia đến bắt rắn vì điều này phụ thuộc vào nhiều yếu tố, bao gồm:\n• Loại rắn: Một số loài rắn hoạt động nhiều hơn vào ban ngày, trong khi những loài khác lại hoạt động nhiều hơn vào ban đêm. Biết được loại rắn bạn đang đối mặt sẽ giúp chuyên gia xác định thời điểm tốt nhất để bắt nó.\n• Vị trí của con rắn: Nếu con rắn đang ở khu vực có nhiều người qua lại hoặc có nguy cơ gây hại cho trẻ em hoặc vật nuôi, tốt nhất bạn nên gọi chuyên gia ngay lập tức.\n• Thời tiết: Rắn thường trở nên hoạt động hơn trong thời tiết mưa và ẩm ướt. Do đó, nếu bạn nhìn thấy rắn vào những ngày này, tốt nhất bạn nên gọi chuyên gia sớm hơn.\n• Lịch trình của chuyên gia: Chuyên gia bắt rắn có thể có lịch trình bận rộn, vì vậy tốt nhất bạn nên gọi cho họ càng sớm càng tốt để đặt lịch hẹn.\nTuy nhiên, nói chung, bạn nên gọi chuyên gia bắt rắn bất cứ khi nào bạn nhìn thấy rắn, đặc biệt là trong những trường hợp sau:\n• Bạn không thể tự xác định loại rắn.\n• Con rắn đang ở khu vực có nhiều người qua lại hoặc có nguy cơ gây hại cho trẻ em hoặc vật nuôi.\n• Bạn cảm thấy không thoải mái hoặc sợ hãi khi tự mình xử lý con rắn.\n• Con rắn có vẻ to lớn hoặc hung dữ.\nDưới đây là một số mẹo để giúp bạn tìm được chuyên gia bắt rắn uy tín:\n• Liên Hệ VSR',
 'Làm sao để loại bỏ an toàn một con rắn không phải rắn độc khỏi tài sản của tôi? - 1. Xác định loại rắn: Xác định xem loài rắn đó có phải là loài rắn không độc hay không. Nếu bạn không chắc chắn, thì không nên cố gắng bắt nó.\n2. Giữ khoảng cách an toàn: Giữ khoảng cách tối thiểu 3 mét với con rắn và không thực hiện bất kỳ động tác đột ngột nào.\n3. Chặn đường thoát: Dùng gậy hoặc cành cây dài để chặn đường thoát của rắn. Đảm bảo rằng bạn không chặn chúng vào góc, vì chúng có thể trở nên hung dữ nếu cảm thấy bị đe dọa.\n4. Dùng dụng cụ tương tác rắn: Nếu bạn có cây kẹp dài hoặc một cây sào dài , hãy sử dụng chúng để từ từ và cẩn thận lấy rắn ra khỏi khu vực. Nhớ giữ phần hân của con rắn để tránh bị cắn. Tuyệt đối nhẹ nhàng, từ từ, tránh gây hoãng loạn và làm rắn bị thương.\n5. Đặt rắn vào hộp đựng: Đặt rắn vào hộp đựng có lỗ thông gió, chẳng hạn như hộp đựng hoặc xô đựng có nắp đậy. Giữ hộp đựng ở nơi an toàn xa khu vực sinh sống của bạn.\n6. Thả rắn ở khu vực thích hợp: Thả rắn ở khu vực thích hợp, cách xa nơi ở của bạn ít nhất 300 mét. Đảm bảo rằng khu vực đó không có người hoặc vật nuôi.\nLưu ý:\n• Khuyến cáo nên bắt buộc liên hệ chuyên gia thuộc VSR để được xử lý an toàn nhất cho cả rắn và người\nKhông nên cố gắng bắt rắn nếu bạn không có kinh nghiệm.\n• Nếu bạn bị rắn cắn, hãy tìm kiếm sự chăm sóc y tế ngay lập tức.\n• Để tránh rắn vào nhà, hãy giữ cho bãi cỏ được cắt ngắn và loại bỏ nơi trú ẩn của rắn, chẳng hạn như đống gỗ hoặc rác.',
 'Có cách nào nhân đạo để đưa những con rắn không mong muốn đi nơi khác không? - Cách nhân đạo để di dời rắn không mong muốn: \nTrước khi di dời:\n• Xác định loại rắn: Việc xác định loại rắn là rất quan trọng để đảm bảo an toàn cho bản thân và lựa chọn phương pháp di dời phù hợp. Một số loại rắn có thể là rắn độc, do đó, việc xác định chính xác là cần thiết.\n• Liên hệ với VSR: Nếu bạn không chắc chắn về cách di dời rắn một cách an toàn, hãy liên hệ với chuyên gia về động vật hoang dã hoặc dịch vụ di dời rắn chuyên nghiệp. Họ có kiến \u200b\u200bthức và chuyên môn để xử lý tình huống một cách an toàn và hiệu quả.\n• Chuẩn bị dụng cụ: Nếu bạn quyết định tự di dời rắn, hãy chuẩn bị các dụng cụ cần thiết như xẻng, thùng nhựa có nắp đậy, găng tay da dày và kính bảo hộ.\n Di dời rắn:\n• Dụ rắn vào thùng: Đảm bảo thùng đủ lớn để chứa rắn thoải mái và có nắp đậy an toàn.\n• Thả rắn ở khu vực an toàn: Mang thùng rắn đến khu vực hoang dã, xa khu dân cư và nguồn nước. Mở nắp thùng và để rắn tự bò ra ngoài.\n• Tránh tiếp xúc trực tiếp: Không bao giờ cố gắng bắt rắn bằng tay trần. Rắn có thể cắn và gây thương tổn nghiêm trọng. \nLưu ý:\n• Không làm rắn bị thương: Tránh giẫm lên, đập hoặc ném rắn.\n• Không sử dụng hóa chất: Hóa chất độc hại có thể gây hại cho rắn và môi trường.\n•     \n• Loại bỏ nơi trú ẩn: Cắt tỉa cây cối và bụi rậm xung quanh nhà. Loại bỏ các đống rác, gỗ hoặc đá có thể cung cấp nơi trú ẩn cho rắn.\n• Tạo rào cản: Lắp đặt hàng rào hoặc lưới thép xung quanh nhà để ngăn rắn xâm nhập.\n• Bịt kín các khe hở: Bịt kín các lỗ hổng và khe nứt trên tường, cửa sổ và nền nhà.\n• \nHãy liên hệ VSR nếu bạn không có đủ chuyên môn và kĩ năng để đảm bảm an toàn cho chính mình và cho con rắn.',
 'Nguy hiểm nào nếu bạn tự ý bắt rắn? - Bạn có thể khiến bản thân mình bị thương do rắn cắn, hoặc làm rắn bị thương hoặc dẫn đến tử vong vì tương tác sai quy chuẩn.',
 'Có hợp pháp không khi di dời một con rắn đến một khu vực khác? - Ở Việt Nam, việc di dời rắn sang khu vực khác là hợp pháp. Rắn là loài động vật hoang dã được bảo vệ theo pháp luật và việc di dời chúng  cần được thực hiện, đánh giá và giám sát bởi các chuyên gia và tổ chức có thẩm quyền',
 'Rắn có thể cảm nhận thế giới xung quanh mà không có tai hay tay chân như thế nào? - Rắn cảm nhận môi trường xung quanh bằng cách sử dụng các giác quan khác như lưỡi, lỗ cảm nhiệt (chỉ một số loài rắn mới có cơ quan này), vảy và đôi mắt.',
 'Rắn có thể nhìn tốt không, hay chúng dựa vào các giác quan khác nhiều hơn? - Chỉ một số loài rắn mới có thể nhìn thấy tốt, nhưng nói chung chúng dựa vào các giác quan khác nhiều hơn là thị giác.',
 'Rắn có biết mùi không, và nếu có, thì chúng dùng khứu giác để làm gì? - Rắn có khả năng ngửi bằng lưỡi, và chúng sử dụng khứu giác của mình theo nhiều cách khác nhau.\n• Theo dõi con mồi: Rắn sử dụng khứu giác để theo dõi con mồi của chúng. Chúng có thể theo dõi mùi hương của con mồi đi xa cả vài km.\n• Tìm bạn tình: Rắn cũng sử dụng khứu giác để tìm bạn tình. Chúng có thể phát hiện ra mùi pheromone của con cái từ rất xa.\n• Tránh kẻ thù: Rắn sử dụng khứu giác của mình để tránh kẻ thù. Chúng có thể phát hiện ra mùi của kẻ thù và trốn tránh chúng.\n• Tìm kiếm thức ăn: Rắn cũng sử dụng khứu giác của mình để tìm kiếm thức ăn. Chúng có thể phát hiện ra mùi của thức ăn từ xa và đi theo mùi hương đó.\n• Giao tiếp: Rắn sử dụng khứu giác của mình để giao tiếp với nhau. Chúng có thể truyền thông tin thông qua mùi pheromone.',
 'Rắn lột da như thế nào, và chúng lột da bao lâu một lần? - Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra. Chúng thường xuyên lột da trong suốt cuộc đời, đặc biệt là khi còn nhỏ. Tần suất lột da phụ thuộc vào loài, tuổi tác, môi trường sống và các yếu tố khác.',
 'Có bất kỳ sự thích nghi đặc biệt nào cho phép loài rắn nuốt con mồi lớn hơn nhiều so với chúng không? - Có nhiều thích nghi đặc biệt giúp rắn có thể nuốt con mồi lớn hơn nhiều so với chính chúng. Ví dụ: rắn có thể mở rộng hàm của mình rất lớn để nuốt con mồi, và chúng cũng có thể tạm dừng quá trình tiêu hóa để tập trung vào việc tiêu hóa con mồi. Ngoài ra, rắn còn có các enzyme tiêu hóa mạnh giúp chúng phân hủy con mồi nhanh chóng. Và thực tế là không phải loài rắn nào cũng có cấu trúc cơ thể phù hợp cho việc nuốt những con mồi ngoại cỡ. Khoa học hiện đại đã khảo sát được rằng rất nhiều các loài rắn chỉ nuốt được con mồi có kích thước bé hoặc to hơn đầu chúng một xíu thôi.',
 'Có bất kỳ mối nguy hiểm nào đối với loài rắn trong thời kỳ ngủ đông của chúng không? - Có nhiều mối nguy hiểm đối với rắn trong suốt thời gian ngủ đông. Chúng có thể trở thành con mồi cho các loài động vật khác như chồn, gấu trúc và cú. Chúng cũng có thể chết vì lạnh hoặc đói nếu chúng không chuẩn bị đủ cho mùa đông.',
 'Rắn sinh sản như thế nào? - Rắn có khoảng thời gian giao phối nhất định theo mùa trong năm, sau đó rắn cái sẽ tìm địa điểm thích hợp và chờ ngày đẻ trứng. Một số loài rắn sẽ có hành vi trông nom trứng cho đến khi rắn con nở, một số thì sẽ bỏ đi sau đi đẻ trứng, còn một số lại có khả năng đẻ ra rắn con sau một thời gian dài mang thai trong bụng.',
 'Rắn đẻ trứng (đẻ trứng) hay đẻ con (sinh ra những con non sống)? - Tuỳ theo mỗi giống rắn mà chúng sẽ có đặc điểm sinh học riêng biệt phục vụ cho hành vi sinh sản đặc thù của bản thân - là đẻ trứng hoặc đẻ con.',
 'Một con rắn thường đẻ bao nhiêu trứng? - Rắn có thể đẻ khoảng 10 đến 100 trứng tuỳ vào chủng loại, mức độ trưởng thành, và kích thước',
 'Liệu rắn bố mẹ có chăm sóc con mình sau khi chúng nở không? - Không, rắn cha mẹ không chăm sóc con của chúng sau khi chúng nở.',
 'Phải mất bao lâu để rắn con độc lập? - Rắn non độc lập ngay từ khi được sinh ra. Bản thân chúng đã là một cổ máy được copy hoàn hảo từ hình thể, tập tính của rắn bố mẹ.',
 'Các loài rắn khác nhau thường ăn gì? - • Rắn hổ mang: chuyên ăn động vật có vú nhỏ, chim và thỉnh thoảng là trứng.\n• Rắn hổ mang chúa: chuyên ăn rắn khác, bao gồm cả chính loài của chúng.\n• Rắn lục: chuyên ăn gặm nhấm và động vật lưỡng cư nhỏ.\n• Rắn nước: chuyên ăn cá, ếch và các sinh vật thủy sinh khác.\n• Rắn cạp nia: chuyên ăn thằn lằn, rắn và động vật gặm nhấm.\n• Rắn trăn và trăn đất: chuyên ăn động vật có vú và chim lớn.\n• Rắn trăn anaconda: chuyên ăn động vật có vú lớn, như lợn, hươu và thậm chí cả cá sấu.\n• Rắn mamba đen: chuyên ăn động vật gặm nhấm, chim và thỏ.\n• Rắn copperhead: chuyên ăn côn trùng, sâu bọ và động vật gặm nhấm nhỏ.\n• Rắn sữa: chuyên ăn động vật gặm nhấm, chim và trứng.',
 'Rắn thường xuyên ăn như thế nào? - Tần suất ăn của rắn phụ thuộc vào nhiều yếu tố như loài, kích thước, nhiệt độ môi trường, tình trạng cơ thể và tình trạng dinh dưỡng. Tuy nhiên, nhìn chung rắn có thể ăn cách nhau từ vài ngày đến vài tháng. Ví dụ: các loài rắn thường ăn từ 5 đến 10 ngày một lần, trong khi các loài rắn lớn hơn có thể ăn cách nhau vài tháng. Ngoài ra, rắn cũng có thể ăn nhiều hơn khi thời tiết ấm hơn hoặc nhịn ăn khi  chuẩn bị sinh sản.',
 'Rắn có bất kỳ cách nào độc đáo để bắt con mồi của mình không? - Có, rắn có thể chủ động đi săn và tìm mồi, hoặc nằm im phục kích chờ mồi. Một số loài rắn đi săn phục kích còn có khả năng dụ mồi bằng cách giả dạng phần chóp đuôi thành miếng mồi đang cựa quậy đầy sống động và hấp dẫn.',
 'Hệ tiêu hóa rắn có thể xử lý con mồi lớn như vậy như thế nào? - Hệ tiêu hóa của rắn có cơ chế đặc biệt để xử lý các con mồi lớn:\n• Dạ dày giãn nở được: Dạ dày của rắn có khả năng giãn nở rất lớn, cho phép chúng nuốt chửng con mồi có kích thước lớn hơn nhiều so với miệng chúng.\n• Dịch tiêu hóa mạnh mẽ: Rắn tiết ra dịch tiêu hóa rất mạnh có chứa enzyme tiêu protein và acid clohydric, giúp phân hủy con mồi thành các chất dinh dưỡng nhỏ hơn.\n• Hàng răng cong ngược: Răng của rắn thường cong ngược vào trong, giúp chúng giữ chặt con mồi và ngăn không cho nó trốn thoát.\n• Quá trình tiêu hóa chậm: Quá trình tiêu hóa ở rắn có thể mất vài ngày hoặc thậm chí vài tuần tùy thuộc vào kích thước của con mồi. Trong thời gian này, rắn thường nghỉ ngơi và không di chuyển nhiều để bảo toàn năng lượng.',
 'Liệu rằng rắn có thể nhịn ăn trong thời gian dài không? - Rắn có thể nhịn đói được một thời gian dài.',
 'Rắn tự vệ trước kẻ săn mồi như thế nào? - Rắn tự vệ trước kẻ thù bằng nhiều cách. Bước đầu tiên và phổ biến nhất là rắn sẽ chạy trốn, khi chạy trốn không thể thành công, Chúng có thể bành trướng cơ thể, cắn, giả chết hoặc phun nọc độc. Một số loài rắn còn có khả năng ngụy trang giúp chúng ẩn nấp khỏi kẻ thù.',
 'Có phải tất cả loài rắn đều sử dụng nọc độc để tự vệ? - Không phải tất cả các loài rắn đều sử dụng nọc độc để tự vệ. Nọc độc được tạo ra với nhiệm vụ chính là giúp rắn đi săn. Tự vệ là cơ chế sự dụng thứ yếu của nọc độc.',
 'Rắn có một số loại chiến lược phòng thủ không dùng nọc độc nào? - Các chiến lược phòng thủ không có nọc độc của rắn\n• Ngụy trang và trốn tránh: Rắn thường ngụy trang tốt hoặc trốn tránh bằng cách ẩn mình trong thảm thực vật, hang hay dưới đất. Chúng cũng có thể thay đổi màu da để hòa hợp với môi trường xung quanh.\n• Tự cắt đuôi: Một số loài rắn, như rắn nước bụng vàng có khả năng tự cắt đuôi để đánh lạc hướng kẻ thù. Đuôi cắt rời sẽ vẫn quằn quại trong vài phút, thu hút sự chú ý của kẻ săn mồi trong khi rắn trốn thoát. Và sau đó con rắn sẽ cụt đuôi mãi mãi chứ không phục hồi đuôi mới như thằn lằn.\n• Phát ra âm thanh đe dọa: Một số loài rắn, chẳng hạn như các loài rắn lục tại khu vực Trung Á, có thể phát ra âm thanh đe dọa bằng cách xoa xát vảy hoặc rung đuôi. Điều này giúp chúng cảnh báo kẻ thù và ngăn chặn chúng tấn công. Bên cạnh có đa số các loài rắn còn lại sẽ dùng tiếng rít to bằng hơi thở của mình để cảnh báo kẻ thù (nhất là rắn hổ mang, rắn lục Russell). Hoặc kinh điển nhất là loài rắn chuông với tháp vảy sếp chồng lên nhau ở chóp đuôi, tạo ra âm thanh khi rung lắc với cường độ cao.\n• Đập đầu: Rắn có thể đập đầu vào kẻ săn mồi như một phương thức tiện tự vệ. Mặc dù không gây ra sát thương, nhưng hành động này có thể gây bất ngờ và khiến kẻ săn phải tháo chạy vì lầm tưởng với một vết cắn tiêm độc.\n• Phun tuyến mùi hôi: Một số loài rắn có thể phun ra chất lỏng hôi từ tuyến hậu môn. Chất lỏng này có thể tấn công mắt hoặc mũi của kẻ săn mồi, khiến chúng khó chịu và lùi lại. Mùi hương hôi thối này sẽ lưu lại rất lâu và khó rửa trôi, có thể khiến kẻ thù bị choáng, hoa mắt, nôn mửa.\n• Phồng cơ thể: Rắn có thể phồng cơ thể để trông to hơn và đe dọa hơn. Điều này khiến chúng trông đáng sợ hơn và có thể ngăn chặn kẻ săn mồi tấn công.',
 'Những con rắn có giả chết như một chiến thuật phòng thủ không? - Loài rắn có phản ứng tự vệ giả chết.',
 'Rắn có giao tiếp với nhau không? - Có, rắn giao tiếp với nhau.',
 'Nếu có thì họ giao tiếp như thế nào? - Bằng cử chỉ ngôn ngữ cơ thể và tiết ra mùi hương đặc biệt từ hậu môn',
 'Liệu có bất kỳ khía cạnh xã hội nào trong hành vi của rắn không? - Hiện tại hành vi xã hội ở rắn vẫn còn đang trong quá trình nghiên cứu',
 'Có một số loài rắn sống theo bầy đàn không? - Không có loài rắn nào sống theo bầy đàn, sự tập hợp bầy đàn chỉ thể hiện qua một số tập tính, vào những thời điểm, của những loài nhất định. Đây không phải hành vi sống phổ biến của rắn.',
 'Con rắn đực và cái tương tác như thế nào trong suốt mùa sinh sản? - Rắn cái sẽ tiết ra mùi hương kêu gọi bạn tình, rắn đực sẽ chủ động tìm đến để giao phối. Nếu các con đực cùng chạm mặt nhau tại một nơi, chúng sẽ chiến đấu với nhau để xem ai được quyền tiếp cận với con cái.',
 'Có loài rắn nào được coi là loài xâm lấn ở những khu vực nhất định không? - Có một số loài rắn được coi là loài xâm lấn ở một số khu vực nhất định. Những loài này là:\nRắn rào nâu (Boiga irregularis): Loài rắn này có nguồn gốc từ Đông Nam Á và đã được đưa vào các đảo Guam, Pohnpei và Saipan. Chúng là loài rắn độc lớn nhất thế giới và gây ra thiệt hại đáng kể cho các loài bản địa.\nTrăn đất(Python bivittatus): Loài trăn này có nguồn gốc từ Đông Nam Á và đã được đưa vào Everglades ở Florida. Chúng là loài rắn không độc nhưng lại gây ra thiệt hại đáng kể cho các loài bản địa, như cá sấu, hươu và chim.\nTrăn gấm (Malayopython reticulatus): Loài trăn này có nguồn gốc từ Đông Nam Á và đã được đưa vào Puerto Rico. Chúng là loài rắn không độc nhưng lại gây ra thiệt hại đáng kể cho nông dân chăn nươi gia súc',
 'Những loài rắn xâm lấn gây ra những vấn đề nào cho hệ sinh thái bản địa? - Chúng gây mất cân bằng hệ sinh thái, vì không có thiên địch nên chúng có thể phát triển quá nhanh, xâm lấn lãnh thổ và nguồn thức ăn của các sinh vật bản địa khác.',
 'Quản lý hoặc kiểm soát quần thể rắn xâm lấn như thế nào? - Các quần thể rắn xâm lấn sẽ được các cơ quan có thẩm quyền xử lý bằng việc di dời vào các tổ chức cứu hộ hoặc tiêu huỷ bằng biện pháp an tử?',
 'Sự hiện diện của rắn xâm lấn có thể ảnh hưởng đến loài người không? - Có, sự hiện diện của loài rắn xâm hại có thể gây ảnh hưởng đến chất lượng cuộc sống của mọi người',
 'Những cân nhắc về đạo đức trong cách ứng phó với các loài rắn xâm lấn là gì? - Liên hệ với chính quyềnd địa phương hoặc các cơ quan ban nghành có thẩm quyền về tự nhiên để có thể đưa ra phương án xử lý phù hợp nhất.',
 'Nhu cầu của các loài rắn cưng khác nhau là gì? - Mỗi một loài rắn sẽ có nguồn gốc xuất xứ và lịch sử khoa học tự nhiên của riêng mình. Đó sẽ là đặc điểm khu vực phân bố, môi trường sống, hành vi săn mồi, con mồi ưa thích, các hành vi có liên quan tới thời gian trong ngày, theo mùa trong năm, theo thời tiết, sinh trưởng, sinh sản, vv.',
 'Làm sao tôi có thể tạo ra môi trường an toàn và lành mạnh cho một con rắn nuôi? - • Tạo môi trường nhiệt độ thích hợp cho rắn. Mỗi loài rắn có nhiệt độ ưa thích riêng, bạn cần tìm hiểu kỹ về loài rắn bạn nuôi để cung cấp môi trường nhiệt độ phù hợp nhất. Bạn có thể sử dụng đèn sưởi hoặc tấm tỏa nhiệt để tạo độ ấm cho rắn.\n• Cung cấp độ ẩm thích hợp. Rắn là loài bò sát cần độ ẩm cao để lột xác và duy trì sức khỏe. Bạn có thể sử dụng máy phun sương hoặc đặt một chậu nước vào trong chuồng rắn để tăng độ ẩm.\n• Cung cấp nơi trú ẩn cho rắn. Rắn cần có nơi ẩn nấp để nghỉ ngơi và cảm thấy an toàn. Bạn có thể sử dụng hang đá, ống nhựa hoặc hộp các tông để làm nơi trú ẩn cho rắn.\n• Cung cấp thức ăn phù hợp. Tùy vào từng loài rắn mà sẽ có chế độ ăn khác nhau. Bạn cần tìm hiểu kỹ về chế độ ăn của loài rắn bạn nuôi để cung cấp thức ăn phù hợp nhất. Bạn có thể cho rắn ăn chuột, thỏ hoặc cá.\n• Vệ sinh chuồng rắn thường xuyên. Chuồng rắn cần được vệ sinh thường xuyên để tránh tích tụ vi khuẩn và nấm mốc, gây hại cho sức khỏe của rắn. Bạn có thể sử dụng xà phòng và nước để vệ sinh chuồng rắn.\n• Đưa rắn đi khám bác sĩ thú y định kỳ. Đưa rắn đi khám bác sĩ thú y định kỳ để kiểm tra sức khỏe và phòng ngừa bệnh tật.',
 'Những loại thức ăn nào thích hợp cho các loài rắn cảnh khác nhau? - Thức ăn cho các loài rắn cảnh khác nhau\nHầu hết các loài rắn cảnh sẽ được nuôi bằng thực phẩm chăn nuôi an toàn, đảm bảo vệ sinh, an toàn thực phẩm cũng như nguồn gốc xuất sứ như chúng ta. Để đảm bảo không mang mầm bệnh, kém chất lượng, gây nguy hiểm đến sức khoẻ vật nuôi. Ví dụ như chuột bạch, chuột lang, thỏ, gà, vịt, ếch, cá, chim cút, vv.',
 'Yêu cầu pháp lý để nuôi một con rắn thú cưng trong khu vực của tôi là gì? - Thú cưng phải được cung cấp từ những shop thú cưng được công khai hoạt động minh bạch, tuyệt đối đảm bảo nguồn gốc vật nuôi từ môi trường sinh sản nhân tạo, không tiếp tay và cố suý cho hành vi khai thác, tàn trữ động vật rừng hoang dã',
 'Tôi có thể tìm thấy những người nuôi có uy tín cho các loài rắn cảnh ở đâu? - Bạn có thể tìm kiếm qua các trang mạng xã hội hoặc google.',
 'Biến đổi khí hậu có thể tác động đến quần thể rắn như thế nào? - Biến đổi khí hậu có thể ảnh hưởng đến quần thể rắn theo nhiều cách. Một số tác động có thể bao gồm:\n• Thay đổi phạm vi phân bố: Nhiệt độ ấm hơn có thể làm cho một số khu vực trở nên thích hợp hơn cho rắn, trong khi những khu vực khác trở nên không phù hợp. Điều này có thể dẫn đến sự thay đổi trong phạm vi phân bố của chúng.\n• Thay đổi số lượng: Nhiệt độ ấm hơn có thể làm tăng tốc độ trao đổi chất của rắn và tăng nhu cầu thức ăn của chúng. Điều này có thể dẫn đến sự gia tăng số lượng của chúng và cạnh tranh nhiều hơn về thức ăn và nơi trú ẩn.\n• Thay đổi hành vi: Nhiệt độ ấm hơn có thể ảnh hưởng đến hành vi của rắn, chẳng hạn như thời gian chúng hoạt động và cách chúng tìm kiếm thức ăn. Điều này có thể ảnh hưởng đến khả năng tồn tại và sinh sản của chúng.\n• Thay đổi tỷ lệ giới tính: Nhiệt độ ấm hơn có thể ảnh hưởng đến tỷ lệ giới tính của rắn, dẫn đến sự gia tăng số cá thể đực hoặc cá thể cái. Điều này có thể ảnh hưởng đến khả năng sinh sản của chúng.\n• Tăng nguy cơ tuyệt chủng: Biến đổi khí hậu có thể làm tăng nguy cơ tuyệt chủng đối với những loài rắn sống trong môi trường cụ thể hoặc có phạm vi phân bố hạn chế.\nTất cả những thay đổi này có thể có tác động đáng kể đến quần thể rắn, dẫn đến sự gia tăng hoặc giảm số lượng, thay đổi phạm vi phân bố và thậm chí là tuyệt chủng.',
 'Những mối đe dọa mà loài rắn phải đối mặt trong tương lai là gì? - Nạn săn bắt rắn hoang dã phục vụ cho việc buôn lậu bất hợp pháp.',
 'Những nỗ lực bảo tồn nào đang được thực hiện để đảm bảo sự tồn tại cho các loài rắn? - Có rất nhiều nỗ lực bảo tồn đang được thực hiện để đảm bảo sự tồn tại của các loài rắn. Các nỗ lực này bao gồm:\n• Bảo vệ môi trường sống: Bảo vệ môi trường sống tự nhiên của rắn là rất quan trọng để đảm bảo sự tồn tại của chúng. Điều này bao gồm việc bảo vệ các khu rừng, đồng cỏ và các khu vực khác nơi rắn sinh sống.\n• Giáo dục nâng cao nhận thức: Giáo dục công chúng về tầm quan trọng của rắn và vai trò của chúng trong hệ sinh thái là rất quan trọng. Điều này sẽ giúp mọi người hiểu được giá trị của rắn và khuyến khích họ bảo vệ chúng.\n• Nghiên cứu và theo dõi: Nghiên cứu đang được tiến hành để tìm hiểu thêm về các loài rắn và nhu cầu của chúng. Điều này sẽ giúp các nhà bảo tồn phát triển các chiến lược hiệu quả hơn để bảo vệ chúng.\n• Chương trình nhân giống nuôi nhốt: Trong một số trường hợp, các chương trình nhân giống nuôi nhốt có thể được sử dụng để bảo tồn các loài rắn đang bị đe dọa.',
 'Làm sao chúng ta có thể thúc đẩy sự hiểu biết và đánh giá cao của công chúng đối với loài rắn? - Thúc đẩy công chúng hiểu và đánh giá cao loài rắn hơn\nRắn thường bị hiểu lầm và sợ hãi, nhưng chúng là một phần quan trọng của hệ sinh thái và có nhiều loài thực sự vô hại. Dưới đây là một số cách để thúc đẩy sự hiểu biết và đánh giá cao của công chúng về loài rắn:\n• Giáo dục trẻ em về rắn. Trẻ em có thể học được về các loại rắn khác nhau, chế độ ăn uống của chúng và vai trò của chúng trong hệ sinh thái.\n• Phân phối các tài liệu giáo dục về rắn. Các tài liệu này có thể bao gồm tờ rơi, sách và bài báo.\n• Tổ chức các buổi nói chuyện và triển lãm về rắn. Những sự kiện này có thể được tổ chức tại địa phương thư viện, trường học hoặc trung tâm cộng đồng.\n• Đưa các chương trình truyền hình và phim ảnh về rắn. Những chương trình này có thể giúp mọi người hiểu được các hành vi và đặc điểm của rắn.\n• Sử dụng phương tiện truyền thông xã hội để nâng cao nhận thức về loài rắn. Có thể đăng bài về loài rắn, chia sẻ ảnh và video về loài rắn và tham gia các nhóm thảo luận về loài rắn.']

In [ ]:
questions = ['Có bất kỳ tín ngưỡng tôn giáo nào liên quan đến loài rắn không?',
 'Tôi có thể ngăn chặn rắn xâm nhập vào tài sản của mình như thế nào?',
 'Tỉ lệ người sợ rắn (ophidiophobia) phổ biến như thế nào?',
 'Rắn đẻ trứng (đẻ trứng) hay đẻ con (sinh ra những con non sống)?',
 'Phải mất bao lâu để phục hồi sau một vết rắn cắn?',
 'Có cách nào nhân đạo để đưa những con rắn không mong muốn đi nơi khác không?',
 'Có mối nguy hiểm nào đối với vật nuôi từ loài rắn thường tìm thấy trong sân không?',
 'Những loại thức ăn nào thích hợp cho các loài rắn cảnh khác nhau?',
 'Những loài rắn xâm lấn gây ra những vấn đề nào cho hệ sinh thái bản địa?',
 'Các loài rắn khác nhau thường ăn gì?']

In [ ]:
embedding_questions = model.encode(questions,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embedding_answers = model.encode(answers)['dense_vecs']
similarity = embedding_questions @ embedding_answers.T

In [ ]:
for i in range(len(questions[:3])):
  # print(f"Question: {questions[i]}")
  print(f"Question: {questions[i]}")
  print()
  print(f"Answer: {answers[similarity[i].argmax()]}")
  print("="*50)

Question: Có bất kỳ tín ngưỡng tôn giáo nào liên quan đến loài rắn không?

Answer: Có bất kỳ tín ngưỡng tôn giáo nào liên quan đến loài rắn không?
Có rất nhiều tín ngưỡng tôn giáo liên quan đến rắn. Trong Ấn Độ giáo, rắn thần Nagaraja được tôn thờ như một vị thần bảo vệ và là biểu tượng của sự tái sinh. Trong Ai Cập cổ đại, thần rắn Apep được coi là hiện thân của bóng tối và hỗn loạn. Trong truyền thống Kitô giáo, rắn là biểu tượng của sự cám dỗ và tội lỗi, nhưng cũng tượng trưng cho sự chữa lành và phục sinh. Trong Phật giáo, rắn là biểu tượng của sự khôn ngoan và từ bi.
Question: Tôi có thể ngăn chặn rắn xâm nhập vào tài sản của mình như thế nào?

Answer: Tôi có thể ngăn chặn rắn xâm nhập vào tài sản của mình như thế nào?
Để ngăn ngừa rắn xâm nhập vào nhà bạn, bạn có thể thực hiện một số biện pháp như sau: - Loại bỏ nguồn thức ăn cho rắn: Rắn thường bị thu hút bởi chuột, gián và các loài gặm nhấm nhỏ khác. Để ngăn chặn rắn xâm nhập, bạn cần loại bỏ nguồn thức ăn của chúng bằng cách b

In [ ]:
top_k = 5
question = "rắn bò vào nhà tôi thì phải làm sao?"
embedding_questions = model.encode(question,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
similarity = embedding_questions @ embedding_answers.T
top_k_unrerank_indices = similarity.argsort()[-top_k:][::-1]

print(f"QUESTION: {question}")
print("="*50)
for i in range(top_k):
  print()
  print(f"ANSWER {i+1} - similarity: {similarity[top_k_unrerank_indices[i]]}\n{answers[top_k_unrerank_indices[i]]}")
  print()
  print("="*50)

QUESTION: rắn bò vào nhà tôi thì phải làm sao?

ANSWER 1 - similarity: 0.7490234375
Tôi có thể ngăn chặn rắn xâm nhập vào tài sản của mình như thế nào?
Để ngăn ngừa rắn xâm nhập vào nhà bạn, bạn có thể thực hiện một số biện pháp như sau: - Loại bỏ nguồn thức ăn cho rắn: Rắn thường bị thu hút bởi chuột, gián và các loài gặm nhấm nhỏ khác. Để ngăn chặn rắn xâm nhập, bạn cần loại bỏ nguồn thức ăn của chúng bằng cách bít kín các lỗ hổng trên tường và sàn nhà, dọn dẹp rác thải và thức ăn thừa, cũng như sử dụng bẫy để diệt chuột. -Lắp đặt hàng rào chắn rắn: Hàng rào chắn rắn là một biện pháp hiệu quả để ngăn chặn rắn xâm nhập vào nhà của bạn. Bạn có thể lắp đặt hàng rào lưới mắt cáo hoặc hàng rào kim loại xung quanh chu vi của ngôi nhà, đảm bảo rằng hàng rào được chôn sâu xuống đất ít nhất 30 cm để ngăn rắn đào hang. - Bít kín các lỗ hổng: Rắn có thể chui qua những lỗ hổng nhỏ, vì vậy bạn cần bít kín tất cả các khoảng trống và lỗ hổng xung quanh nhà, đặc biệt là ở các khu vực như đường ống, 

## Set up Rerank Component

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=False) # Setting use_fp16 to True speeds up computation with a slight performance degradation

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [ ]:
import numpy as np

rerank_scores = reranker.compute_score([[question, answers[top_k_unrerank_indices[i]]] for i in range(top_k)], normalize=True)
rerank_scores = np.array(rerank_scores)
print(rerank_scores)

[0.96873892 0.93934823 0.95398667 0.96003227 0.38973114]


In [ ]:
top_k_rerank_indices = rerank_scores.argsort()[-top_k:][::-1]

print(f"QUESTION: {question}")
print("="*50)
for i in range(top_k):
  print()
  print(f"ANSWER {i+1} - score: {rerank_scores[top_k_rerank_indices[i]]}\n{answers[top_k_unrerank_indices[top_k_rerank_indices[i]]]}")
  print()
  print("="*50)

QUESTION: rắn bò vào nhà tôi thì phải làm sao?

ANSWER 1 - score: 0.9687389241401998
Tôi có thể ngăn chặn rắn xâm nhập vào tài sản của mình như thế nào?
Để ngăn ngừa rắn xâm nhập vào nhà bạn, bạn có thể thực hiện một số biện pháp như sau: - Loại bỏ nguồn thức ăn cho rắn: Rắn thường bị thu hút bởi chuột, gián và các loài gặm nhấm nhỏ khác. Để ngăn chặn rắn xâm nhập, bạn cần loại bỏ nguồn thức ăn của chúng bằng cách bít kín các lỗ hổng trên tường và sàn nhà, dọn dẹp rác thải và thức ăn thừa, cũng như sử dụng bẫy để diệt chuột. -Lắp đặt hàng rào chắn rắn: Hàng rào chắn rắn là một biện pháp hiệu quả để ngăn chặn rắn xâm nhập vào nhà của bạn. Bạn có thể lắp đặt hàng rào lưới mắt cáo hoặc hàng rào kim loại xung quanh chu vi của ngôi nhà, đảm bảo rằng hàng rào được chôn sâu xuống đất ít nhất 30 cm để ngăn rắn đào hang. - Bít kín các lỗ hổng: Rắn có thể chui qua những lỗ hổng nhỏ, vì vậy bạn cần bít kín tất cả các khoảng trống và lỗ hổng xung quanh nhà, đặc biệt là ở các khu vực như đường ống,

## Set up LLM Component with Gemini

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [ ]:
GEMINI = genai.GenerativeModel('gemini-pro', safety_settings = safety_settings)

In [ ]:
class DummyAnswer():
  def __init__(self, text):
    self.text = text

def join_list_into_string(my_list):
    result = ""
    for i, item in enumerate(my_list):
        result += f"{i+1}. {item}\n\n"
    return result.rstrip()


def separate_last_user_query(conversation):
    lines = conversation.splitlines()

    if not lines:
        return None

    user_queries = [line.split(": ", maxsplit=1)[1] for line in lines if line.startswith("User:")]

    return user_queries[-1] if user_queries else None

def choose_method_for_handling_user_query(conversation_text):

    user_last_query = separate_last_user_query(conversation_text)
    prompt = f"""Bạn là một Trợ lý và bạn có một cơ sở dữ liệu về rắn.
Cho cuộc trò chuyện sau đây:
{conversation_text}

Câu hỏi của người dùng:
{user_last_query}

Để trả lời cho Câu hỏi của người dùng một cách hữu ích và chính xác, bạn cần cần làm gì trong 3 việc sau?
A. Trả lời trực tiếp câu hỏi nếu bạn đã biết cách trả lời (áp dụng cho lời chào, cuộc trò chuyện thông thường, v.v.)
B. Lấy thông tin từ cơ sở dữ liệu và trả lời dựa trên nó
Lưu ý: Vui lòng trả lời bằng chữ cái (A. hoặc B)
"""

    response = GEMINI.generate_content(prompt)
    return response


def ask_for_clarification_questions(conversation_text):
    prompt = f"""Dựa trên cuộc trò chuyện sau đây
{conversation_text}

Hãy đặt ra câu hỏi để làm rõ ý người dùng.
"""
    response = GEMINI.generate_content(prompt)
    return response


def query_reformulation(conversation_text):
    user_last_query = separate_last_user_query(conversation_text)
    prompt = f"""Dựa trên cuộc trò chuyện sau đây
{conversation_text}

Câu hỏi của người dùng:
{user_last_query}

Hãy viết lại câu hỏi của người dùng sao cho câu hỏi đã được viết lại có thể được sử dụng để tìm kiếm và đáp ứng nhu cầu thông tin của người dùng. Câu hỏi hỏi viết lại phải là một câu hỏi.
    """

    response = GEMINI.generate_content(prompt)
    return response


def answer_user_directly(conversation_text):
    prompt = f"""Bạn là một trợ lý
Dựa trên cuộc trò chuyện sau đây
{conversation_text}

Hãy trả lời một cách chính xác và thành thực nhất
Câu trả lời:
    """

    response = GEMINI.generate_content(prompt)
    return response


def check_if_context_is_relevant(query, string_context):
    prompt = f"""Cho câu hỏi và thông tin sau, hãy cho biết liệu những thông tin đó có đủ thể trả lời được câu hỏi
Câu hỏi: {query}

Thông tin: {string_context}

Chọn:
A. Có
B. Không
Lưu ý chỉ trả lời bằng chữ (A. hoặc B.)
"""
    response = GEMINI.generate_content(prompt)
    if "A." in response.text or "A" == response.text: return True
    else: return False


def answer_query_with_context(query, conversation_text, contexts):
    string_context = join_list_into_string(contexts)

    context_relevant = check_if_context_is_relevant(query, string_context)

    if not context_relevant: return DummyAnswer("Tôi không có đủ thông tin để trả lời câu hỏi của bạn.")

    else:
        print(f"\n--------\nRETRIEVED INFO: {string_context}\n--------\n")
        prompt = f"""Bạn là một trợ lý
Cho lịch sử cuộc trò chuyện, câu hỏi người dùng và thông tin bên ngoài

Lịch sử cuộc trò chuyện:
{conversation_text}

Câu hỏi người dùng:
{query}

Thông tin:
{string_context}

Hãy sử dụng thông tin để trả lời câu hỏi của người dùng.
        """
        response = GEMINI.generate_content(prompt)
        return response

def handle_conversation_turn(conversation_history):
    method_for_answering = choose_method_for_handling_user_query(conversation_history)

    if "A." in method_for_answering.text or "A" == method_for_answering.text:
        print("\n--------\nLOGGING: answer query directly\n--------\n")
        return answer_user_directly(conversation_history)

    elif "B." in method_for_answering.text or "B" == method_for_answering.text:
        reformulated_query = query_reformulation(conversation_history)
        print("\n--------\nLOGGING: answer query with retrieval. Reformulated query:", reformulated_query.text, "\n--------\n")
        embedding_questions = model.encode(reformulated_query.text,
                            batch_size=12,
                            max_length=8192,
                            )['dense_vecs']
        top_k = 5
        similarity = embedding_questions @ embedding_answers.T
        top_k_unrerank_indices = similarity.argsort()[-top_k:][::-1]
        rerank_scores = reranker.compute_score([[question, answers[top_k_unrerank_indices[i]]] for i in range(top_k)], normalize=True)
        rerank_scores = np.array(rerank_scores)
        top_k_rerank_indices = [top_k_unrerank_indices[i] for i in rerank_scores.argsort()[-top_k:][::-1]]
        contexts = [answers[top_k_rerank_indices[i]] for i in range(top_k)]

        return answer_query_with_context(reformulated_query.text, conversation_history, contexts)

    elif "C." in method_for_answering.text or "C" == method_for_answering.text:
        print("\n--------\nLOGGING: asking clarification question\n--------\n")
        return ask_for_clarification_questions(conversation_history)


def chat():
  """Simulates a simple chat interface with conversation history in Google Colab."""
  conversation_history = ""
  while True:
    user_input = input("User: ")
    conversation_history += f"User: {user_input}\n"

    if user_input.lower() == "quit":
      print("Assistant: Goodbye!")
      break

    response_conv_turn = handle_conversation_turn(conversation_history)
    response = response_conv_turn.text
    conversation_history += f"Assistant: {response}\n"
    print(f"Assistant: {response}\n")
  return conversation_history

In [ ]:
chattext = chat()